# Inference

This script showcases the different models available in fishbAIT and how to use them efficiently.

In [1]:
import torch
import pandas as pd
import numpy as np
from inference_utils.fishbAIT_for_inference import fishbAIT_for_inference

Specify the model version and load the model

In [2]:
MODEL_VERSION = 'EC10_invertebrates'

In [3]:
fishbait = fishbAIT_for_inference(model_version=MODEL_VERSION)
fishbait.load_fine_tuned_model()

Load the SMILES you wish to predict

In [4]:
data = pd.read_excel('../data/tutorials/Inference_example_2.xlsx')
data

,SMILES,cmpdname
0,CC(=O)Oc1ccccc1C(O)=O,Aspirin
1,[Cr],Chromium
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,Fluoxetine hydrochloride
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,Clofenotane
4,[Cu],Copper
...,...,...
995,[Pb++].[O-]c1c(cc(c([O-])c1[N+]([O-])=O)[N+]([...,Lead styphnate
996,CC(C)(C)C(O)(CCc1ccc(Cl)cc1)Cn2cncn2,Tebuconazole
997,[Na+].[Na+].[Na+].[Na+].OCCN(CCO)c1nc(Nc2ccc(c...,OpticalBrightenerBbu220
998,CNC.OC(=O)COc1ccc(Cl)cc1Cl,"2,4-D dimethylamine salt"


Specify the endpoint and effect you wish to predict and make the prediction

In [5]:
PREDICTION_ENDPOINT = 'EC10'
PREDICTION_EFFECT = 'MOR'
EXPOSURE_DURATION = 96
SMILES_COLUMN_NAME = 'SMILES'

In [6]:
fishbait.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].iloc[0:10].tolist(), exposure_duration=EXPOSURE_DURATION, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT)

Renamed NOEC *EC10* in 0 positions
Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.


100%|██████████| 2/2 [00:00<00:00,  9.03it/s]


,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L)
0,CC(=O)Oc1ccccc1C(O)=O,96,EC10,MOR,CC(=O)Oc1ccccc1C(=O)O,-16.496807,3.185612e-17
1,[Cr],96,EC10,MOR,[Cr],-16.133186,7.358913e-17
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,96,EC10,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-17.033726,9.252823e-18
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,96,EC10,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-18.201427,6.288869e-19
4,[Cu],96,EC10,MOR,[Cu],-15.602854,2.495435e-16
5,CCNc1nc(Cl)nc(NC(C)C)n1,96,EC10,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,-16.651567,2.230656e-17
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,96,EC10,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,-15.855769,1.393898e-16
7,CC(Br)(CO)[N+]([O-])=O,96,EC10,MOR,CC(Br)(CO)[N+](=O)[O-],-14.331802,4.657980e-15
8,c1ccc2c(c1)c3cccc4cccc2c34,96,EC10,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-15.887511,1.295653e-16
9,[Cl-].[Cl-].[Zn++],96,EC10,MOR,[Cl-].[Cl-].[Zn+2],-15.117763,7.624958e-16


If you run into truble using this model use the `__help__` function

In [7]:
fishbait.__help__()


        This is a python class used to load and use the fine-tuned deep-learning model `fishbAIT` for environmental toxicity predictions in fish algae and aquatic invertebrates.
        The models have been trained on a large corpus of SMILES (chemical representations) on data collected from various sources.

        Currently there are nine models available for use. The models are divided by toxicity endpoint and by species group. The models are the following:
        **Fish**
        - `EC50_fish` The EC50 model is trained on EC50 mortality (MOR) data and is thus suitable for the prediction of said endpoints.
        - `EC10_fish` The EC10 model is trained on EC10/NOEC data with various effects (mortality, intoxication, development, reproduction, morphology, growth and population) ab. (MOR, ITX, DVP, REP, MPH, GRO, POP)
        - `EC50EC10_fish` The EC50EC10 model is trained on EC50, EC10 and NOEC data with various effects (mortality, intoxication, development, reproduction, morphol